<a href="https://colab.research.google.com/github/theresaltrinh/project_voting/blob/main/voting_model_building_tori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Voting Project -- Model Building by Tori and Kelsey

In [36]:
! git clone https://github.com/theresaltrinh/project_voting

fatal: destination path 'project_voting' already exists and is not an empty directory.


In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [38]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

vadf = pd.read_csv('/content/drive/My Drive/4th Year/IntroML/DS3001 Projects/Voting Project/cleaned_voting_VA.csv', index_col=0)
cdf = pd.read_csv('/content/drive/My Drive/4th Year/IntroML/DS3001 Projects/Voting Project/cleaned_county_adjacencies.csv', index_col=0)
df18 = pd.read_csv("./project_voting/data/county_data/0002_ds239_20185_county_E.csv", encoding = "latin1")
df20 = pd.read_csv("./project_voting/data/county_data/0002_ds249_20205_county_E.csv", encoding = "latin1")

Mounted at /content/drive


<ipython-input-38-85905b95be6a>:7: DtypeWarning: Columns (3,4,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,

In [48]:
cdf.rename(columns={'CountyCode':'countyCode'}, inplace=True)

In [39]:
vadf.head()

,year,countyName,countyCode,candidate,party,candidatevotes,totalvotes
0,2000,Accomack,51001,Al Gore,Democrat,5092,11925
1,2000,Accomack,51001,George W. Bush,Republican,6352,11925
2,2000,Accomack,51001,Ralph Nader,Green,220,11925
3,2000,Accomack,51001,Other,Other,261,11925
4,2000,Albemarle,51003,Al Gore,Democrat,16255,36846


In [40]:
df18.head()

,GISJOIN,YEAR,STUSAB,REGIONA,DIVISIONA,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,...,AJ1CE007,AJ1SE001,AJ1TE001,AJ1TE002,AJ1TE003,AJ1UE001,AJ1UE002,AJ1UE003,AJ3EE001,AJ3QE001
0,GIS Join Match Code,Data File Year,State Postal Abbreviation,Region Code,Division Code,State Name,State Code,County Name,County Code,County Subdivision Code,...,Not in labor force,Total,Total,Occupied,Vacant,Total,Owner occupied,Renter occupied,Median gross rent,Median value (dollars)
1,G0100010,2014-2018,AL,NaN,NaN,Alabama,01,Autauga County,001,NaN,...,17632,23315,23315,21115,2200,21115,15814,5301,966,147900
2,G0100030,2014-2018,AL,NaN,NaN,Alabama,01,Baldwin County,003,NaN,...,69682,111945,111945,78622,33323,78622,57881,20741,958,189800
3,G0100050,2014-2018,AL,NaN,NaN,Alabama,01,Barbour County,005,NaN,...,11310,11937,11937,9186,2751,9186,5640,3546,590,92900
4,G0100070,2014-2018,AL,NaN,NaN,Alabama,01,Bibb County,007,NaN,...,9713,9161,9161,6840,2321,6840,5135,1705,714,96500


In [41]:
df20.head()

,GISJOIN,YEAR,STUSAB,REGIONA,DIVISIONA,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,...,AMWSE004,AMWSE005,AMWSE006,AMWSE007,AMWSE008,AMWSE009,AMWSE010,AMWSE011,AMWSE012,AMWSE013
0,GIS Join Match Code,Data File Year,State Postal Abbreviation,Region Code,Division Code,State Name,State Code,County Name,County Code,County Subdivision Code,...,With an Internet subscription: Broadband of an...,With an Internet subscription: Cellular data plan,With an Internet subscription: Cellular data p...,With an Internet subscription: Broadband such ...,With an Internet subscription: Broadband such ...,With an Internet subscription: Satellite Inter...,With an Internet subscription: Satellite Inter...,With an Internet subscription: Other service w...,Internet access without a subscription,No Internet access
1,G0100010,2016-2020,AL,NaN,NaN,Alabama,01,Autauga County,001,NaN,...,17831,15390,2666,13780,2004,1695,349,57,402,3307
2,G0100030,2016-2020,AL,NaN,NaN,Alabama,01,Baldwin County,003,NaN,...,71498,65644,13937,53139,4634,8874,788,0,2149,10018
3,G0100050,2016-2020,AL,NaN,NaN,Alabama,01,Barbour County,005,NaN,...,6025,5351,1800,3620,547,933,79,0,563,2700
4,G0100070,2016-2020,AL,NaN,NaN,Alabama,01,Bibb County,007,NaN,...,5526,5074,1889,2669,307,1182,142,0,210,1520


In [42]:
# Getting data just for Viginia
df18v = df18[df18['STATE'] == 'Virginia']
df20v = df20[df20['STATE'] == 'Virginia']

In [43]:
# Create combined df
df = pd.concat([df18v, df20v], ignore_index = True)
df.head()

,GISJOIN,YEAR,STUSAB,REGIONA,DIVISIONA,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,...,AMWSE004,AMWSE005,AMWSE006,AMWSE007,AMWSE008,AMWSE009,AMWSE010,AMWSE011,AMWSE012,AMWSE013
0,G5100010,2014-2018,VA,NaN,NaN,Virginia,51,Accomack County,001,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,G5100030,2014-2018,VA,NaN,NaN,Virginia,51,Albemarle County,003,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,G5100050,2014-2018,VA,NaN,NaN,Virginia,51,Alleghany County,005,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,G5100070,2014-2018,VA,NaN,NaN,Virginia,51,Amelia County,007,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,G5100090,2014-2018,VA,NaN,NaN,Virginia,51,Amherst County,009,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


AMPWE001 - AMPWE010 refers to AMPWE001, AMPWE002, ..., AMPWE010. In general, the first column (e.g. AMPWE001) refers to 'Total', not the breakdown of the variables, so we do not include this one. For more information, refer to the codebook.

AMPWE002 - AMPWE010: Race

AMP4E002 - AMP4E003: Ethnicity (Hispanic or Latino Origin)

AMRZE002 - AMRZE025: Educational attainment for population 25 years or older

AMR8E001: Median household income in the last 12 months

In [44]:
# Getting only variables of interest
var_names = [
    "GISJOIN", # GIS Join Match Code
    "YEAR",
    "COUNTY",
    "COUNTYA", # County code
    "AMPWE002", "AMPWE003", "AMPWE004", "AMPWE005", "AMPWE006",
    "AMPWE007", "AMPWE008", "AMPWE009", "AMPWE010",  # Race
    "AMP4E002", "AMP4E003",  # Ethnicity
    "AMRZE002", "AMRZE003", "AMRZE004", "AMRZE005", "AMRZE006",
    "AMRZE007", "AMRZE008", "AMRZE009", "AMRZE010", "AMRZE011",
    "AMRZE012", "AMRZE013", "AMRZE014", "AMRZE015", "AMRZE016",
    "AMRZE017", "AMRZE018", "AMRZE019", "AMRZE020", "AMRZE021",
    "AMRZE022", "AMRZE023", "AMRZE024", "AMRZE025",  # Educational attainment
    "AMR8E001"  # Income
]

df_new = df[var_names]
df_new.head()

,GISJOIN,YEAR,COUNTY,COUNTYA,AMPWE002,AMPWE003,AMPWE004,AMPWE005,AMPWE006,AMPWE007,...,AMRZE017,AMRZE018,AMRZE019,AMRZE020,AMRZE021,AMRZE022,AMRZE023,AMRZE024,AMRZE025,AMR8E001
0,G5100010,2014-2018,Accomack County,001,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,G5100030,2014-2018,Albemarle County,003,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,G5100050,2014-2018,Alleghany County,005,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,G5100070,2014-2018,Amelia County,007,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,G5100090,2014-2018,Amherst County,009,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# Renaming columns
new_var_names = {
    "COUNTYA": "countyCode",
    "AMPWE002": "white",
    "AMPWE003": "black",
    "AMPWE004": "american_indian_and_alaska_native",
    "AMPWE005": "asian",
    "AMPWE006": "native_hawaiian_and_pacific_islander",
    "AMPWE007": "other",
    "AMPWE008": "two_or_more_races",
    "AMPWE009": "two_or_more_races_including_some_other_race",
    "AMPWE010": "three_or_more_races_excluding_some_other_race", # Race
    "AMP4E002": "not_hispanic_or_latino",
    "AMP4E003": "hispanic_or_latino", # Ethnicity
    "AMRZE002": "no_schooling_completed",
    "AMRZE003": "nursery_school",
    "AMRZE004": "kindergarten",
    "AMRZE005": "1st_grade",
    "AMRZE006": "2nd_grade",
    "AMRZE007": "3rd_grade",
    "AMRZE008": "4th_grade",
    "AMRZE009": "5th_grade",
    "AMRZE010": "6th_grade",
    "AMRZE011": "7th_grade",
    "AMRZE012": "8th_grade",
    "AMRZE013": "9th_grade",
    "AMRZE014": "10th_grade",
    "AMRZE015": "11th_grade",
    "AMRZE016": "12th_grade_or_no_diploma",
    "AMRZE017": "high_school_diploma",
    "AMRZE018": "GED",
    "AMRZE019": "some_college_under_1year",
    "AMRZE020": "some_college_over_1year",
    "AMRZE021": "associate_degree",
    "AMRZE022": "bachelor_degree",
    "AMRZE023": "master_degree",
    "AMRZE024": "professional_school_degree",
    "AMRZE025": "doctorate_degree", # Education level
    "AMR8E001": "income"
}

df_renamed = df_new.rename(columns = new_var_names)

In [46]:
df_renamed.isnull().sum()

GISJOIN                                            0
YEAR                                               0
COUNTY                                             0
countyCode                                         0
white                                            133
black                                            133
american_indian_and_alaska_native                133
asian                                            133
native_hawaiian_and_pacific_islander             133
other                                            133
two_or_more_races                                133
two_or_more_races_including_some_other_race      133
three_or_more_races_excluding_some_other_race    133
not_hispanic_or_latino                           133
hispanic_or_latino                               133
no_schooling_completed                           133
nursery_school                                   133
kindergarten                                     133
1st_grade                                     

I tried melting the dataframe to have race, ethnicity, and education to be columns, but it was producing a weird error. So I'm just going to keep it like this.

I'm also replacing all NaN values with 0's because that's what NaN's in this case actually mean. Also need to do this so we can run sklearn models.

Note: Add in conclusion that if we wanted to take this further, we could find a better way of handling NaN values.

In [47]:
df_final = df_renamed.replace(np.nan, 0)
df_final.isnull().sum() # Looks good

GISJOIN                                          0
YEAR                                             0
COUNTY                                           0
countyCode                                       0
white                                            0
black                                            0
american_indian_and_alaska_native                0
asian                                            0
native_hawaiian_and_pacific_islander             0
other                                            0
two_or_more_races                                0
two_or_more_races_including_some_other_race      0
three_or_more_races_excluding_some_other_race    0
not_hispanic_or_latino                           0
hispanic_or_latino                               0
no_schooling_completed                           0
nursery_school                                   0
kindergarten                                     0
1st_grade                                        0
2nd_grade                      

At this point we have 2 main dataframes, df_final (with 18 and 20 data) and vadf.

## Notes from office hours

Subtract Republican from Democrat and predict margin

By what margin will one party beat the other (instead of winner)

M files -- margin of error that census thinks corresponds to that estimate

Pick one with interesting variables and go with that

If there is something we want to do but can't figure it out, it is OK to write it up in the report and move on (conclusion)

In [57]:
# Ensure COUNTYA is a string, pad with leading zeros if necessary, and prepend '51'
df_final['countyCode'] = df_final['countyCode'].astype(str).str.zfill(3).apply(lambda x: '51' + x)
df_final['countyCode'].head()

0    51001
1    51003
2    51005
3    51007
4    51009
Name: countyCode, dtype: object

In [64]:
# Convert countyCode to string in both dfs
vadf['countyCode'] = vadf['countyCode'].astype(int)
df_final['countyCode'] = df_final['countyCode'].astype(int)

In [66]:
# Outer join to find unmatched entries
combined_df = pd.merge(vadf, df_final, on='countyCode', how='outer', indicator=True)
combined_df.tail()

,year,countyName,countyCode,candidate,party,candidatevotes,totalvotes,GISJOIN,YEAR,COUNTY,...,GED,some_college_under_1year,some_college_over_1year,associate_degree,bachelor_degree,master_degree,professional_school_degree,doctorate_degree,income,_merge
5331,2020,Winchester city,51840,Jo Jorgensen,Libertarian,213,12113,G5108400,2016-2020,Winchester city,...,859.0,1013.0,2734.0,1307.0,3777.0,1930.0,451.0,239.0,61102.0,both
5332,2020,Winchester city,51840,Joseph R Biden Jr,Democrat,6610,12113,G5108400,2014-2018,Winchester city,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
5333,2020,Winchester city,51840,Joseph R Biden Jr,Democrat,6610,12113,G5108400,2016-2020,Winchester city,...,859.0,1013.0,2734.0,1307.0,3777.0,1930.0,451.0,239.0,61102.0,both
5334,2020,Winchester city,51840,Other,Other,69,12113,G5108400,2014-2018,Winchester city,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
5335,2020,Winchester city,51840,Other,Other,69,12113,G5108400,2016-2020,Winchester city,...,859.0,1013.0,2734.0,1307.0,3777.0,1930.0,451.0,239.0,61102.0,both


In [67]:
# Find entries that exist in vadf but not in df_final
unmatched_vadf = combined_df[combined_df['_merge'] == 'left_only']

# Find entries that exist in df_final but not in vadf
unmatched_df_final = combined_df[combined_df['_merge'] == 'right_only']

print("Unmatched in vadf:")
print(unmatched_vadf[['countyCode', '_merge']])

print("\nUnmatched in df_final:")
print(unmatched_df_final[['countyCode', '_merge']])

Unmatched in vadf:
      countyCode     _merge
3840       51515  left_only
3841       51515  left_only
3842       51515  left_only
3843       51515  left_only
3844       51515  left_only
3845       51515  left_only
3846       51515  left_only
3847       51515  left_only
3848       51515  left_only
3849       51515  left_only
3850       51515  left_only
3851       51515  left_only
3852       51515  left_only
3853       51515  left_only
3854       51515  left_only
3855       51515  left_only

Unmatched in df_final:
Empty DataFrame
Columns: [countyCode, _merge]
Index: []


There were county codes in vadf that didn't have corresponding entries in df_final, so there were unmatched rows that create null values in the merged dataset -- I took care of this above.

In [75]:
rows_with_51515 = vadf[vadf['countyCode'] == 51515]
rows_with_51515.head()

,year,countyName,countyCode,candidate,party,candidatevotes,totalvotes
384,2000,Bedford,51515,Al Gore,Democrat,1078,2441
385,2000,Bedford,51515,George W. Bush,Republican,1269,2441
386,2000,Bedford,51515,Ralph Nader,Green,38,2441
387,2000,Bedford,51515,Other,Other,56,2441
824,2004,Bedford,51515,John Kerry,Democrat,1042,2542


It looks like Bedford is throwing things off, so remove it

In [74]:
# Dropping rows where countyCode is '51515' from vadf
vadf_clean = vadf[vadf['countyCode'] != 51515]
combined_df_clean = combined_df[combined_df['countyCode'] != 51515]
print("After dropping, counts are:", vadf_clean['countyCode'].value_counts())

After dropping, counts are: countyCode
51001    20
51177    20
51540    20
51530    20
51520    20
         ..
51081    20
51079    20
51077    20
51075    20
51840    20
Name: count, Length: 133, dtype: int64


In [76]:
combined_df_clean.isnull().sum()

year                                             0
countyName                                       0
countyCode                                       0
candidate                                        0
party                                            0
candidatevotes                                   0
totalvotes                                       0
GISJOIN                                          0
YEAR                                             0
COUNTY                                           0
white                                            0
black                                            0
american_indian_and_alaska_native                0
asian                                            0
native_hawaiian_and_pacific_islander             0
other                                            0
two_or_more_races                                0
two_or_more_races_including_some_other_race      0
three_or_more_races_excluding_some_other_race    0
not_hispanic_or_latino         

In [78]:
# Dropping 'COUNTY' because it is the same as 'countyName'
combined_df_clean.drop(columns = ['COUNTY'], inplace = True)

<ipython-input-78-03f101362c88>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df_clean.drop(columns = ['COUNTY'], inplace = True)


In [79]:
print("year:", combined_df_clean['year'].unique())
print("YEAR:", combined_df_clean['YEAR'].unique())

# Changing 'YEAR' to 'election_cycle' to be more accurate
combined_df_clean.rename(columns = {"YEAR": "election_cycle"}, inplace = True)

year: [2000 2004 2008 2012 2016 2020]
YEAR: ['2014-2018' '2016-2020']


<ipython-input-79-2b388bfee7c9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df_clean.rename(columns = {"YEAR": "election_cycle"}, inplace = True)


In [80]:
combined_df_clean.head()

,year,countyName,countyCode,candidate,party,candidatevotes,totalvotes,GISJOIN,election_cycle,white,...,GED,some_college_under_1year,some_college_over_1year,associate_degree,bachelor_degree,master_degree,professional_school_degree,doctorate_degree,income,_merge
0,2000,Accomack,51001,Al Gore,Democrat,5092,11925,G5100010,2014-2018,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
1,2000,Accomack,51001,Al Gore,Democrat,5092,11925,G5100010,2016-2020,21773.0,...,1246.0,1616.0,2512.0,2044.0,2697.0,1440.0,299.0,257.0,46178.0,both
2,2000,Accomack,51001,George W. Bush,Republican,6352,11925,G5100010,2014-2018,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
3,2000,Accomack,51001,George W. Bush,Republican,6352,11925,G5100010,2016-2020,21773.0,...,1246.0,1616.0,2512.0,2044.0,2697.0,1440.0,299.0,257.0,46178.0,both
4,2000,Accomack,51001,Ralph Nader,Green,220,11925,G5100010,2014-2018,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both


In [81]:
# Grouping the votes by year and party and summing them up
party_votes = combined_df_clean.groupby(['year', 'party'])['candidatevotes'].sum().unstack()
party_votes['margin'] = party_votes['Democrat'] - party_votes['Republican']

# Reset index if needed to flatten the DataFrame
party_votes.reset_index(inplace=True)

In [84]:
# Merging margin into combined_df_clean
combined_df_clean = pd.merge(combined_df_clean, party_votes[['year', 'margin']], on='year', how='left')
combined_df_clean.head()

,year,countyName,countyCode,candidate,party,candidatevotes,totalvotes,GISJOIN,election_cycle,white,...,some_college_under_1year,some_college_over_1year,associate_degree,bachelor_degree,master_degree,professional_school_degree,doctorate_degree,income,_merge,margin
0,2000,Accomack,51001,Al Gore,Democrat,5092,11925,G5100010,2014-2018,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both,-440528.0
1,2000,Accomack,51001,Al Gore,Democrat,5092,11925,G5100010,2016-2020,21773.0,...,1616.0,2512.0,2044.0,2697.0,1440.0,299.0,257.0,46178.0,both,-440528.0
2,2000,Accomack,51001,George W. Bush,Republican,6352,11925,G5100010,2014-2018,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both,-440528.0
3,2000,Accomack,51001,George W. Bush,Republican,6352,11925,G5100010,2016-2020,21773.0,...,1616.0,2512.0,2044.0,2697.0,1440.0,299.0,257.0,46178.0,both,-440528.0
4,2000,Accomack,51001,Ralph Nader,Green,220,11925,G5100010,2014-2018,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both,-440528.0


In [88]:
combined_df_clean.tail(3)

,year,countyName,countyCode,candidate,party,candidatevotes,totalvotes,GISJOIN,election_cycle,white,...,some_college_under_1year,some_college_over_1year,associate_degree,bachelor_degree,master_degree,professional_school_degree,doctorate_degree,income,_merge,margin
5317,2020,Winchester city,51840,Joseph R Biden Jr,Democrat,6610,12113,G5108400,2016-2020,21617.0,...,1013.0,2734.0,1307.0,3777.0,1930.0,451.0,239.0,61102.0,both,902276.0
5318,2020,Winchester city,51840,Other,Other,69,12113,G5108400,2014-2018,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both,902276.0
5319,2020,Winchester city,51840,Other,Other,69,12113,G5108400,2016-2020,21617.0,...,1013.0,2734.0,1307.0,3777.0,1930.0,451.0,239.0,61102.0,both,902276.0


In [86]:
# Check for null values in 'margin' after merging
print("Null values in 'margin':", combined_df_clean['margin'].isnull().sum())

Null values in 'margin': 0


In [89]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

X = party_votes[['year']]  # Predictor
y = party_votes['margin']  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE: {rmse}')

predicted_margin_2024 = model.predict([[2024]])
print(f'Predicted margin for 2024: {predicted_margin_2024[0]}')

RMSE: 589864.4883936637
Predicted margin for 2024: 879490.0


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


If predicted margin is pos, it indicates a Democratic win --  model predicts that Democrats will receive more votes than Republicans by that margin.

If predicted margin is neg, it indicates a Republican win --  model predicts that Republicans will receive more votes than Democrats by the absolute value of that margin.

###Done cleaning, now model building